In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load and preprocess the dataset
data = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images = training_images / 255.0
test_images = test_images / 255.0

# Define the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Custom callback to track unchanged weights
class WeightsTracker(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        # Save the initial weights
        self.previous_weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]
        self.unchanged_weights = [[np.zeros_like(w) for w in layer_weights] for layer_weights in self.previous_weights]

    def on_epoch_end(self, epoch, logs=None):
        current_weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]
        print(f"\nWeights comparison after epoch {epoch + 1}:")

        # Compare current weights with previous weights and track unchanged weights
        for layer_idx, (prev_w, curr_w) in enumerate(zip(self.previous_weights, current_weights)):
            print(f"Layer {layer_idx}:")
            for w_idx, (prev, curr) in enumerate(zip(prev_w, curr_w)):
                unchanged_mask = prev == curr  # Create a mask of unchanged weights
                unchanged_count = np.sum(unchanged_mask)
                total_count = unchanged_mask.size

                # Track unchanged weights for each sub-layer
                self.unchanged_weights[layer_idx][w_idx] = unchanged_mask

                print(f"  Sub-layer {w_idx} - Unchanged Weights: {unchanged_count}/{total_count}")

        # Update previous weights to current for the next epoch comparison
        self.previous_weights = current_weights

    def on_train_end(self, logs=None):
        print("\nFinal unchanged weights mask at the end of training:")
        for layer_idx, weights in enumerate(self.unchanged_weights):
            print(f"Layer {layer_idx}:")
            for w_idx, mask in enumerate(weights):
                unchanged_count = np.sum(mask)
                total_count = mask.size
                print(f"  Sub-layer {w_idx} - Unchanged Weights: {unchanged_count}/{total_count}")

# Instantiate the callback
weights_tracker = WeightsTracker()

# Train the model with the custom callback
model.fit(training_images, training_labels, epochs=10, callbacks=[weights_tracker])

# Evaluate the model
model.evaluate(test_images, test_labels)



# input layer = 28 * 28 = 784
# layer 1 = Dense = 128
#         No. of weights from input layer to layer 1 = 784 * 128 = 1,00,352
#
# layer 2 = Dense = 10
#         No. of weights from layer 1 to layer 2 = 128 * 10 = 1,280

#
# starting mein random initialize hua
# fir 1st epoch mein
#
#
#
#
#
#
#
#
#
#

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1865/1875 [============================>.] - ETA: 0s - loss: 0.2520 - accuracy: 0.9287
Weights comparison after epoch 1:
Layer 0:
  Sub-layer 0 - Unchanged Weights: 10532/100352
  Sub-layer 1 - Unchanged Weights: 0/128
Layer 1:
  Sub-layer 0 - Unchanged Weights: 0/1280
  Sub-layer 1 - Unchanged Weights: 0/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2517 - accuracy: 0.9288
Epoch 2/10
1869/1875 [============================>.] - ETA: 0s - loss: 0.1100 - accuracy: 0.9675
Weights comparison after epoch 2:
Layer 0:
  Sub-layer 0 - Unchanged Weights: 11568/100352
  Sub-layer 1 - Unchanged Weights: 0/128
Layer 1:
  Sub-layer 0 - Unchanged Weights: 0/1280
  Sub-layer 1 - Unchanged Weights: 0/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1101 - accuracy: 0.9675
Epoch 3/10
1868/1875 [============================>.] - ETA: 0s - loss: 0.0747 - accuracy: 0.9772
Weights compa

[0.08371984958648682, 0.9772999882698059]